# Salary Prediction
Predicting whether an individual in the 1994 US Census earns over $50k based on their information.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Get Dataset

In [3]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets 
  
print(X.head())

   age         workclass  fnlwgt  education  education-num  \
0   39         State-gov   77516  Bachelors             13   
1   50  Self-emp-not-inc   83311  Bachelors             13   
2   38           Private  215646    HS-grad              9   
3   53           Private  234721       11th              7   
4   28           Private  338409  Bachelors             13   

       marital-status         occupation   relationship   race     sex  \
0       Never-married       Adm-clerical  Not-in-family  White    Male   
1  Married-civ-spouse    Exec-managerial        Husband  White    Male   
2            Divorced  Handlers-cleaners  Not-in-family  White    Male   
3  Married-civ-spouse  Handlers-cleaners        Husband  Black    Male   
4  Married-civ-spouse     Prof-specialty           Wife  Black  Female   

   capital-gain  capital-loss  hours-per-week native-country  
0          2174             0              40  United-States  
1             0             0              13  United-St

## Data Exploration

In [4]:
X.isnull().sum()

age                 0
workclass         963
fnlwgt              0
education           0
education-num       0
marital-status      0
occupation        966
relationship        0
race                0
sex                 0
capital-gain        0
capital-loss        0
hours-per-week      0
native-country    274
dtype: int64

In [5]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       47879 non-null  object
 2   fnlwgt          48842 non-null  int64 
 3   education       48842 non-null  object
 4   education-num   48842 non-null  int64 
 5   marital-status  48842 non-null  object
 6   occupation      47876 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capital-gain    48842 non-null  int64 
 11  capital-loss    48842 non-null  int64 
 12  hours-per-week  48842 non-null  int64 
 13  native-country  48568 non-null  object
dtypes: int64(6), object(8)
memory usage: 5.2+ MB


In [6]:
print(X["education"].value_counts())
print(X["education-num"].value_counts())

# drop education since it's a duplicate feature - use the "education-num" feature as it is already close to a good encoding
X.drop("education", axis=1, inplace=True)

HS-grad         15784
Some-college    10878
Bachelors        8025
Masters          2657
Assoc-voc        2061
11th             1812
Assoc-acdm       1601
10th             1389
7th-8th           955
Prof-school       834
9th               756
12th              657
Doctorate         594
5th-6th           509
1st-4th           247
Preschool          83
Name: education, dtype: int64
9     15784
10    10878
13     8025
14     2657
11     2061
7      1812
12     1601
6      1389
4       955
15      834
5       756
8       657
16      594
3       509
2       247
1        83
Name: education-num, dtype: int64


C:\Users\Adam\AppData\Local\Temp\ipykernel_19912\4283663453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop("education", axis=1, inplace=True)


In [7]:
X["fnlwgt"].value_counts()

203488    21
120277    19
190290    19
125892    18
126569    18
          ..
286983     1
185942     1
234220     1
214706     1
350977     1
Name: fnlwgt, Length: 28523, dtype: int64

In [8]:
X["occupation"].value_counts()

Prof-specialty       6172
Craft-repair         6112
Exec-managerial      6086
Adm-clerical         5611
Sales                5504
Other-service        4923
Machine-op-inspct    3022
Transport-moving     2355
Handlers-cleaners    2072
?                    1843
Farming-fishing      1490
Tech-support         1446
Protective-serv       983
Priv-house-serv       242
Armed-Forces           15
Name: occupation, dtype: int64

In [9]:
# plt.hist(df['column1'], bins=range(min(df['column1']), max(df['column1']) + 1, 1), edgecolor='black')
# plt.title('Distribution of Column1')
# plt.xlabel('Value')
# plt.ylabel('Frequency')
# plt.show()

In [10]:
X_test = X[y["income"].str.endswith(".")]
y_test = y[y["income"].str.endswith(".")]
X_train = X[y["income"].str.endswith("K")]
y_train = y[y["income"].str.endswith("K")]
print(y.value_counts())

income
<=50K     24720
<=50K.    12435
>50K       7841
>50K.      3846
dtype: int64


## Preprocess Data

### Missing Value Imputation

In [11]:
class CustomImputer:
    data = None
    col_to_impute = None
    cols_to_use = None

    def __init__(self):
        pass

    def fit_custom_mode_imputer(self, df: pd.DataFrame, col_to_impute: str, cols_to_use: [str]):
        self.col_to_impute = col_to_impute
        self.cols_to_use = cols_to_use

        self.data = df.groupby(self.cols_to_use)[self.col_to_impute].agg(lambda x: pd.Series.mode(x)[0]).reset_index()
        self.data = self.data.rename(columns={self.col_to_impute: "agg_col"})
    
    def apply_imputer(self, df):
        new_df = df.merge(self.data, how="left", on=self.cols_to_use)
        new_df[self.col_to_impute] = np.where(new_df[self.col_to_impute].isna(), new_df["agg_col"], new_df[self.col_to_impute])
        new_df.drop("agg_col", axis=1, inplace=True)
        return new_df


workclass

In [12]:
# Impute using mode distribution, based on nearest matching individuals
ci = CustomImputer()
ci.fit_custom_mode_imputer(X_train, "workclass", ["education-num", "race", "sex"])

# Only the test set is missing values
X_test = ci.apply_imputer(X_test)

X_test.isna().sum()


age                 0
workclass           0
fnlwgt              0
education-num       0
marital-status      0
occupation        966
relationship        0
race                0
sex                 0
capital-gain        0
capital-loss        0
hours-per-week      0
native-country    274
dtype: int64

occupation

In [13]:
ci = CustomImputer()
ci.fit_custom_mode_imputer(X_train, "occupation", ["education-num", "workclass", "sex"])

# Only the test set is missing values
X_test = ci.apply_imputer(X_test)

# Replace remaining missing values with "?" as mode matches can't be found
X_test["occupation"] = X_test["occupation"].replace(np.nan, "?")

X_test.isna().sum()

age                 0
workclass           0
fnlwgt              0
education-num       0
marital-status      0
occupation          0
relationship        0
race                0
sex                 0
capital-gain        0
capital-loss        0
hours-per-week      0
native-country    274
dtype: int64

native-country

In [14]:
ci = CustomImputer()
ci.fit_custom_mode_imputer(X_train, "native-country", ["race", "education-num", "occupation"])

# Only the test set is missing values
X_test = ci.apply_imputer(X_test)

# Replace remaining missing values with "?" as mode matches can't be found
X_test["native-country"] = X_test["native-country"].replace(np.nan, "?")

X_test.isna().sum()

age               0
workclass         0
fnlwgt            0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
dtype: int64

### Scaling + Normalisation
Apply minmax or z-score to all numerical columns.

In [15]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32561 entries, 0 to 32560
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education-num   32561 non-null  int64 
 4   marital-status  32561 non-null  object
 5   occupation      32561 non-null  object
 6   relationship    32561 non-null  object
 7   race            32561 non-null  object
 8   sex             32561 non-null  object
 9   capital-gain    32561 non-null  int64 
 10  capital-loss    32561 non-null  int64 
 11  hours-per-week  32561 non-null  int64 
 12  native-country  32561 non-null  object
dtypes: int64(6), object(7)
memory usage: 3.5+ MB


In [16]:
# min-max scaling
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler().fit(X_train[["age", "fnlwgt", "education-num", "capital-gain", "capital-loss", "hours-per-week"]])
X_train[["age", "fnlwgt", "education-num", "capital-gain", "capital-loss", "hours-per-week"]] = mms.transform(X_train[["age", "fnlwgt", "education-num", "capital-gain", "capital-loss", "hours-per-week"]])
X_test[["age", "fnlwgt", "education-num", "capital-gain", "capital-loss", "hours-per-week"]] = mms.transform(X_test[["age", "fnlwgt", "education-num", "capital-gain", "capital-loss", "hours-per-week"]])

C:\Users\Adam\AppData\Local\Temp\ipykernel_19912\2352589296.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[["age", "fnlwgt", "education-num", "capital-gain", "capital-loss", "hours-per-week"]] = mms.transform(X_train[["age", "fnlwgt", "education-num", "capital-gain", "capital-loss", "hours-per-week"]])


### Variable Encoding

In [17]:
# One hot encoding
X_train = pd.get_dummies(X_train, columns = ["workclass", "marital-status", "occupation", "relationship", "race", "sex"], drop_first=True)
X_test = pd.get_dummies(X_test, columns = ["workclass", "marital-status", "occupation", "relationship", "race", "sex"], drop_first=True)
X_train.head()

# Handle a case where a column is missing due to a value being missing in the test set
missing_column = list(set(X_train.columns) - set(X_test.columns))[0]
X_test[missing_column] = 0
X_test = X_test[X_train.columns]

In [18]:
X_train["native-country"] = np.where(X_train["native-country"] == "United-States", 1, 0)
X_test["native-country"] = np.where(X_test["native-country"] == "United-States", 1, 0)
X_train.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,native-country,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,race_Asian-Pac-Islander,race_Black,race_Other,race_White,sex_Male
0,0.301370,0.044302,0.800000,0.02174,0.0,0.397959,1,0,0,0,...,1,0,0,0,0,0,0,0,1,1
1,0.452055,0.048238,0.800000,0.00000,0.0,0.122449,1,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,0.287671,0.138113,0.533333,0.00000,0.0,0.397959,1,0,0,0,...,1,0,0,0,0,0,0,0,1,1
3,0.493151,0.151068,0.400000,0.00000,0.0,0.397959,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
4,0.150685,0.221488,0.800000,0.00000,0.0,0.397959,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0


In [19]:
y_test = pd.get_dummies(y_test, columns=["income"], drop_first=True)
y_train = pd.get_dummies(y_train, columns=["income"], drop_first=True)
print(y_test.value_counts())
print(y_train.value_counts())

income_>50K.
0               12435
1                3846
dtype: int64
income_>50K
0              24720
1               7841
dtype: int64


## Feature Selection
Use Recursive Feature Elimination

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV

estimator = RandomForestClassifier()
selector = RFECV(estimator, step=1, cv=3)
selector = selector.fit(X_train, y_train.values.ravel())

In [21]:
X_train = selector.transform(X_train)
X_test = selector.transform(X_test)

## Modelling

### GLM
Use the statsmodels library to implement our GLM.

In [22]:
import statsmodels.api as sm

model = sm.GLM(np.asarray(y_train), np.asarray(X_train), family=sm.families.Binomial())
fitted_model = model.fit()

print(fitted_model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                32561
Model:                            GLM   Df Residuals:                    32518
Model Family:                Binomial   Df Model:                           42
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -10596.
Date:                Mon, 30 Oct 2023   Deviance:                       21191.
Time:                        17:48:37   Pearson chi2:                 4.30e+05
No. Iterations:                    22   Pseudo R-squ. (CS):             0.3644
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             1.5558      0.116     13.399      0.0

In [23]:
y_pred = fitted_model.predict(np.asarray(X_test))

from sklearn.metrics import classification_report

threshold = 0.5 
y_pred_binary = [1 if pred >= threshold else 0 for pred in y_pred]

print(classification_report(y_test, y_pred_binary))

              precision    recall  f1-score   support

           0       0.88      0.93      0.91     12435
           1       0.73      0.60      0.66      3846

    accuracy                           0.85     16281
   macro avg       0.81      0.77      0.78     16281
weighted avg       0.85      0.85      0.85     16281



### Gradient Boosting Machines
Gradient Boosting Classifier

In [26]:
from sklearn.ensemble import GradientBoostingClassifier

classifier = GradientBoostingClassifier().fit(X_train, y_train)

classifier.score(X_test, y_test)

c:\ProgramData\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8706467661691543

AdaBoost

In [27]:
from sklearn.ensemble import AdaBoostClassifier

classifier = AdaBoostClassifier().fit(X_train, y_train)

classifier.score(X_test, y_test)

c:\ProgramData\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.861924943185308

XGBoost

In [28]:
from xgboost import XGBClassifier

classifier = XGBClassifier().fit(X_train, y_train)

classifier.score(X_test, y_test)

0.8716295067870524

Note: I have not tuned the hyperparameters, and am using the default parameters present in the classifiers. To do this, I would use a ParameterGrid to test a range of parameters and select the best performing ones. 